## 필요한 라이브러리 준비

In [ ]:
!pip install numpy scipy librosa unidecode inflect librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 16.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy.io.wavfile import write

##tacotron2 모델 준비

In [ ]:
import torch
tacotron2 = torch.hub.load('nvidia/DeepLearningExamples', 'nvidia_tacotron2')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/nvidia/DeepLearningExamples/zipball/master" to /root/.cache/torch/hub/master.zip
/root/.cache/torch/hub/nvidia_DeepLearningExamples_master/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/roo

Tacotron2(
  (embedding): Embedding(148, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0-2): 3 x Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (linear_layer): Linear(in_features=80, out_features=256, bias=False)
        )
        (1): LinearNorm(
          (linear_layer): Linear(in_features=256, out_features=256, bias=False)
        )
      )
    )
    (attention_rnn): LSTMCell(768, 1024)
    (attention_layer): Attention(
      (query_layer): LinearNorm(
        (linear_layer): Linear(in_features=1024, out_features=128, bias=False)
      )
      (memory_layer): LinearNorm(
        (linear_layer): Linear(in_fea

## WaveGlow 모델 준비

In [ ]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip


WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0-3): 4 x WN(
      (in_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0-6): 7 x Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
        (7

##스크립트 불러와서 음성 합성하기

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving output1.txt to output1.txt


In [ ]:
with open('output1.txt') as file_object:  #열고자 하는 파일을 입력하세요
    contents = file_object.read()
text_preprocess = contents.split('.')

In [ ]:
import os
downloads_folder = os.path.join(os.getcwd(), "output")
if not os.path.exists(downloads_folder):
  os.makedirs(downloads_folder)

os.chdir('/content/output')

In [ ]:
for i in range(len(text_preprocess)-1):
  utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
  sequences, lengths = utils.prepare_input_sequence([text_preprocess[i]]) #문장 단위로 불러오기
  
  with torch.no_grad(): #합성하기
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
    audio_numpy = audio[0].data.cpu().numpy()
    rate = 25000

    from scipy.io.wavfile import write #파일로 저장
    write("text"+str(i+1)+".wav", rate, audio_numpy)

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/nvidia_DeepLearningExamples_master/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/text/__init__.py:74: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return s in _symbol_to_id and s is not '_' and s is not '~'
/root/.cache/torch/hub/nvidia_DeepLearningExamples_master/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/text/__init__.py:74: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return s in _symbol_to_id and s is not '_' and s is not '~'
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root

##저장

In [ ]:
from google.colab import files
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!zip -r /content/wave_output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/text9.wav (deflated 6%)
  adding: content/output/text8.wav (deflated 6%)
  adding: content/output/text7.wav (deflated 6%)
  adding: content/output/text5.wav (deflated 6%)
  adding: content/output/text2.wav (deflated 6%)
  adding: content/output/text6.wav (deflated 6%)
  adding: content/output/text10.wav (deflated 6%)
  adding: content/output/text1.wav (deflated 6%)
  adding: content/output/text4.wav (deflated 6%)
  adding: content/output/text3.wav (deflated 6%)


In [ ]:
files.download('/content/wave_output.zip')  #다운받을 파일 이름 입력

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#바로 확인
#from IPython.display import Audio
#Audio(audio_numpy, rate=rate)